In [6]:
#demonstration __new__ called before __init__
class MyClass:
    def __new__(cls, *args, **kwargs):
            print("new() is called", cls, args, kwargs)
            instance = super().__new__(cls)
            print(f" instance {instance}")
            return instance

    def __init__(self, attribute):
        print('__init__()', self, attribute)
        self.attribute = attribute

In [7]:
myclass = MyClass("Bonjour")

new() is called <class '__main__.MyClass'> ('Bonjour',) {}
 instance <__main__.MyClass object at 0x0000013BCF40FC80>
__init__() <__main__.MyClass object at 0x0000013BCF40FC80> Bonjour


In [19]:
#demonstration uses case of __new__
class CountInstanceClass:
    instance_created = 0
    def __new__(cls, *args, **kwargs):
            print("new() is called", cls, args, kwargs)
            instance = super().__new__(cls)
            print(f" instance {instance}")
            cls.instance_created += 1
            return instance

    def __init__(self, attribute):
        print('__init__()', self, attribute)
        self.attribute = attribute
        
myclass = CountInstanceClass("Bonjour") 
myclass2 = CountInstanceClass("Bonjour") 
myclass3 = CountInstanceClass("Bonjour") 
print(CountInstanceClass.instance_created)

new() is called <class '__main__.CountInstanceClass'> ('Bonjour',) {}
 instance <__main__.CountInstanceClass object at 0x0000013BCF56A930>
__init__() <__main__.CountInstanceClass object at 0x0000013BCF56A930> Bonjour
new() is called <class '__main__.CountInstanceClass'> ('Bonjour',) {}
 instance <__main__.CountInstanceClass object at 0x0000013BCF46E030>
__init__() <__main__.CountInstanceClass object at 0x0000013BCF46E030> Bonjour
new() is called <class '__main__.CountInstanceClass'> ('Bonjour',) {}
 instance <__main__.CountInstanceClass object at 0x0000013BCF46FC50>
__init__() <__main__.CountInstanceClass object at 0x0000013BCF46FC50> Bonjour
3


In [28]:
#demonstration uses case of __new__
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls)
        return cls._instance
a = Singleton()
b = Singleton()
print(a == b) #True
print(a)
print(b)

True


In [21]:
#demonstration each class is a type
print(type(myclass3))

<class '__main__.CountInstanceClass'>


In [22]:
a = int(3)

In [23]:
print(type(a))

<class 'int'>


In [24]:
print(type(CountInstanceClass))

<class 'type'>


In [25]:
print(type.__class__.__mro__)

(<class 'type'>, <class 'object'>)


In [26]:
class MyClass:
    def methode(self):
        return 1
m = MyClass()
print(type(m))
print(type(MyClass))
print(m.__dict__)
print(MyClass.__dict__)

<class '__main__.MyClass'>
<class 'type'>
{}
{'__module__': '__main__', 'methode': <function MyClass.methode at 0x0000013BCF4CB9C0>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}


In [27]:
def methode(self):
    return 1
MyClass2 = type("MyClass2", (object,), {"methode" : methode})
m = MyClass2()
print(type(m))
print(type(MyClass2))
print(m.__dict__)
print(MyClass2.__dict__)

<class '__main__.MyClass2'>
<class 'type'>
{}
{'methode': <function methode at 0x0000013BCF4CB600>, '__module__': '__main__', '__dict__': <attribute '__dict__' of 'MyClass2' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass2' objects>, '__doc__': None}


In [32]:
class RevealingMeta(type):
    
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print(mcs, "__prepare__ called")
        return super().__prepare__(name, bases, **kwargs)
        
    def __new__(mcs, name, bases, namespace, **kwargs):
        print(mcs, "__new__ called")
        return super().__new__(mcs, name, bases, namespace)
        
    def __init__(cls, name, bases, namespace, **kwargs):
        print(cls, "__init__ called")
        super().__init__(name, bases, namespace)
        
    def __call__(cls, *args, **kwargs):
        print(cls, "__call__ called")
        return super().__call__(*args, **kwargs)

class MyClass(metaclass = RevealingMeta):
    def __new__(cls, *args, **kwargs):
            print("new() is called", cls, args, kwargs)
            instance = super().__new__(cls)
            return instance

    def __init__(self, attribute):
        print('__init__()', self, attribute)
        self.attribute = attribute


<class '__main__.RevealingMeta'> __prepare__ called
<class '__main__.RevealingMeta'> __new__ called
<class '__main__.MyClass'> __init__ called


In [33]:
myinstance = MyClass("OK")

<class '__main__.MyClass'> __call__ called
new() is called <class '__main__.MyClass'> ('OK',) {}
__init__() <__main__.MyClass object at 0x0000013BCF4410D0> OK


In [51]:
class PluginMeta(type):
    plugins = {}
    def __new__(mcs, name, bases, namespace):
            namespace["Vendredi"] = lambda :print("Bonjuour")
            cls = super().__new__(mcs, name, bases, namespace)
            if name != 'PluginBase':
                    # Enregistrement automatique du plugin
                    PluginMeta.plugins[name] = cls
           
            return cls
    def __init__(cls, name, bases, namespace, **kwargs):
        print(cls, "__init__ called")
        super().__init__(name, bases, namespace)
        
        
class PluginBase(metaclass=PluginMeta):
    pass
class HealthPlugin(PluginBase):
    pass
class InventoryPlugin(PluginBase):
    pass
# Les plugins sont automatiquement enregistrés et peuvent être accédés
print(PluginMeta.plugins)  # Affiche les plugins enregistrés

<class '__main__.PluginBase'> __init__ called
<class '__main__.HealthPlugin'> __init__ called
<class '__main__.InventoryPlugin'> __init__ called
{'HealthPlugin': <class '__main__.HealthPlugin'>, 'InventoryPlugin': <class '__main__.InventoryPlugin'>}


In [52]:
print(HealthPlugin.__dict__)
h = HealthPlugin()
print(h.__dict__)

{'__module__': '__main__', 'Vendredi': <function PluginMeta.__new__.<locals>.<lambda> at 0x0000013BCFAFD120>, '__doc__': None}
{}


In [54]:
print(HealthPlugin.Vendredi())


Bonjuour
None


AttributeError: 'HealthPlugin' object has no attribute 'Vendredi'